**ReAct** is a design pattern used in AI agents that stands for *Reasoning + Acting*. It allows a language model (LLM) to interleave internal reasoning (Thought) with external actions (like tool use) in a structured multi-step process.<br>
**Source:** https://docs.langchain.com/oss/python/langchain/agents

In [1]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

In [2]:
from langchain_core.tools import tool
import requests

@tool
def get_weather_data(city:str) -> dict:
    """
    This function fetches current weather data for a given city.
    """

    url = f"https://wttr.in/{city}?format=j1"

    response = requests.get(url).json()["current_condition"][0]
    
    return response

In [3]:
tools = [search_tool,get_weather_data]

In [4]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="ministral-3:3b",
    temperature=0,
).bind_tools(tools)

In [5]:
from langchain.agents import create_agent
agent = create_agent(
        model=llm,
        tools=tools,
        system_prompt="You are a helpful assistant who writes crisp answer",
    )

inputs = {"messages": [{"role": "user", "content": "Find the capital of Bihar, then find its current weather"}]}
for chunk in agent.stream(inputs, stream_mode="updates"):
    print(chunk)

{'model': {'messages': [AIMessage(content='The capital of **Bihar** is **Patna**.\n\n', additional_kwargs={}, response_metadata={'model': 'ministral-3:3b', 'created_at': '2026-01-30T17:47:26.6004728Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1470608700, 'load_duration': 127082600, 'prompt_eval_count': 159, 'prompt_eval_duration': 216277300, 'eval_count': 26, 'eval_duration': 1122387800, 'logprobs': None, 'model_name': 'ministral-3:3b', 'model_provider': 'ollama'}, id='lc_run--019c1004-5447-7453-a8de-fdc107652449-0', tool_calls=[{'name': 'get_weather_data', 'args': {'city': 'Patna'}, 'id': '3f473942-9a9b-4887-b192-3ed2096671da', 'type': 'tool_call'}], usage_metadata={'input_tokens': 159, 'output_tokens': 26, 'total_tokens': 185})]}}
{'tools': {'messages': [ToolMessage(content='{"FeelsLikeC": "17", "FeelsLikeF": "63", "cloudcover": "25", "humidity": "77", "localObsDateTime": "2026-01-30 10:12 PM", "observation_time": "04:42 PM", "precipInches": "0.0", "precipMM": "0.0", "p